### Prerequsite - Upgrade Scikit Learn
The current workspace has scikit-learn v0.19.1 installed. However, you can upgrade scikit-learn to 0.24.x. and use this [OneHotEncoder](https://scikit-learn.org/0.21/modules/generated/sklearn.preprocessing.OneHotEncoder.html) library. 


In [ ]:
#import sklearn
#print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
#import os
#os.environ['PATH'] = f"{os.environ['PATH']}:/root/.local/bin"

In [ ]:
# Restart the Kernel. 

#import IPython

#IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

In [ ]:
#!python -m pip install --upgrade scikit-learn
#import sklearn
#print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
# Now you can import and use OneHotEncoder
#from sklearn.preprocessing import OneHotEncoder
# your code goes here

In [ ]:
# Similarly, should you need any other package, they can install it as:
#!python -m pip install 'tensorflow-tensorboard<0.2.0,>=0.1.0'

# Project: Identify Customer Segments

In this project, you will apply unsupervised learning techniques to identify segments of the population that form the core customer base for a mail-order sales company in Germany. These segments can then be used to direct marketing campaigns towards audiences that will have the highest expected rate of returns. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

This notebook will help you complete this task by providing a framework within which you will perform your analysis steps. In each step of the project, you will see some text describing the subtask that you will perform, followed by one or more code cells for you to complete your work. **Feel free to add additional code and markdown cells as you go along so that you can explore everything in precise chunks.** The code cells provided in the base template will outline only the major tasks, and will usually not be enough to cover all of the minor tasks that comprise it.

It should be noted that while there will be precise guidelines on how you should handle certain tasks in the project, there will also be places where an exact specification is not provided. **There will be times in the project where you will need to make and justify your own decisions on how to treat the data.** These are places where there may not be only one way to handle the data. In real-life tasks, there may be many valid ways to approach an analysis task. One of the most important things you can do is clearly document your approach so that other scientists can understand the decisions you've made.

At the end of most sections, there will be a Markdown cell labeled **Discussion**. In these cells, you will report your findings for the completed section, as well as document the decisions that you made in your approach to each subtask. **Your project will be evaluated not just on the code used to complete the tasks outlined, but also your communication about your observations and conclusions at each stage.**

In [ ]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Additional Libraries:

# Path
from pathlib import Path


# magic word for producing visualizations in notebook
%matplotlib inline


# Additional Magic Words:
# magic word for reloading modules before executing code
%load_ext autoreload
%autoreload 2


'''
Import note: The classroom currently uses sklearn version 0.19.
If you need to use an imputer, it is available in sklearn.preprocessing.Imputer,
instead of sklearn.impute as in newer versions of sklearn.
'''
pd.options.display.max_rows = None
pd.options.display.max_columns = None

### Step 0: Load the Data

There are four files associated with this project (not including this one):

- `Udacity_AZDIAS_Subset.csv`: Demographics data for the general population of Germany; 891211 persons (rows) x 85 features (columns).
- `Udacity_CUSTOMERS_Subset.csv`: Demographics data for customers of a mail-order company; 191652 persons (rows) x 85 features (columns).
- `Data_Dictionary.md`: Detailed information file about the features in the provided datasets.
- `AZDIAS_Feature_Summary.csv`: Summary of feature attributes for demographics data; 85 features (rows) x 4 columns

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. You will use this information to cluster the general population into groups with similar demographic properties. Then, you will see how the people in the customers dataset fit into those created clusters. The hope here is that certain clusters are over-represented in the customers data, as compared to the general population; those over-represented clusters will be assumed to be part of the core userbase. This information can then be used for further applications, such as targeting for a marketing campaign.

To start off with, load in the demographics data for the general population into a pandas DataFrame, and do the same for the feature attributes summary. Note for all of the `.csv` data files in this project: they're semicolon (`;`) delimited, so you'll need an additional argument in your [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call to read in the data properly. Also, considering the size of the main dataset, it may take some time for it to load completely.

Once the dataset is loaded, it's recommended that you take a little bit of time just browsing the general structure of the dataset and feature summary file. You'll be getting deep into the innards of the cleaning in the first major step of the project, so gaining some general familiarity can help you get your bearings.

In [ ]:
from WGU_D499_P2_DCook.dataset import load_csv_with_parameters

# Load in the general demographics data.
azdias_general_demographics_original_df = load_csv_with_parameters("Udacity_AZDIAS_Subset.csv", sep=";", na_values=np.nan)

# Load in the feature summary file.
features_summary_original_df = load_csv_with_parameters("AZDIAS_Feature_Summary.csv", sep=";")


In [ ]:
# Check the structure of the data after it's loaded (e.g. print the number of
# rows and columns, print the first few rows).

# Priting the shape of the general demographics dataframe
print(f"AZDIAS General Demographics shape: {azdias_general_demographics_original_df.shape}")

# Printing the shape of the features summary dataframe
print(f"AZDIAS Feature Summary shape: {features_summary_original_df.shape}")


AZDIAS General Demographics Details:

In [ ]:
azdias_general_demographics_original_df.info()

In [ ]:
azdias_general_demographics_original_df.describe()

In [ ]:
azdias_general_demographics_original_df.head(10)

AZDIAS Feature Summary Details:

In [ ]:
features_summary_original_df.info()

In [ ]:
features_summary_original_df.describe()

In [ ]:
features_summary_original_df.head(10)

> **Tip**: Add additional cells to keep everything in reasonably-sized chunks! Keyboard shortcut `esc --> a` (press escape to enter command mode, then press the 'A' key) adds a new cell before the active cell, and `esc --> b` adds a new cell after the active cell. If you need to convert an active cell to a markdown cell, use `esc --> m` and to convert to a code cell, use `esc --> y`. 

## Step 1: Preprocessing

### Step 1.1: Assess Missing Data

The feature summary file contains a summary of properties for each demographics data column. You will use this file to help you make cleaning decisions during this stage of the project. First of all, you should assess the demographics data in terms of missing data. Pay attention to the following points as you perform your analysis, and take notes on what you observe. Make sure that you fill in the **Discussion** cell with your findings and decisions at the end of each step that has one!

#### Step 1.1.1: Convert Missing Value Codes to NaNs
The fourth column of the feature attributes summary (loaded in above as `feat_info`) documents the codes from the data dictionary that indicate missing or unknown data. While the file encodes this as a list (e.g. `[-1,0]`), this will get read in as a string object. You'll need to do a little bit of parsing to make use of it to identify and clean the data. Convert data that matches a 'missing' or 'unknown' value code into a numpy NaN value. You might want to see how much data takes on a 'missing' or 'unknown' code, and how much data is naturally missing, as a point of interest.

**As one more reminder, you are encouraged to add additional cells to break up your analysis into manageable chunks.**

In [ ]:
# Identify missing or unknown data values and convert them to NaNs.

# Total Number of existing Missing/Null Values in the dataset or Natural Nulls
azdias_general_demographics_original_df.isnull().sum()

In [ ]:
# Initializing a dictionary variable to hold the count of natural nulls for each column
nat_null_count = {}

# For Loop to iterate through each column in the azdias_general_demographics_original_df
# and calculate the total number of natural nulls (NaN values) in each column.
# Storing the results in the nat_null_count dictionary.
for column in azdias_general_demographics_original_df.columns:
    
    column_total_nat_nulls = azdias_general_demographics_original_df[column].isna().sum()
    nat_null_count[column] = column_total_nat_nulls

In [ ]:
# For Loop to iterate through the nat_null_count dictionary
# and print the column name along with the total number of natural nulls in descending order.

for column_name, nat_null_count in sorted(nat_null_count.items(), key=lambda item: item[1], reverse=True):
    #print(column_name, null_count)
    if nat_null_count > 0:
        print(f" Column Name: '{column_name}' has a total of '{nat_null_count:,}' natural null/missing values.")

print(f"\nAll other columns have '0' natural null/missing values")

In [ ]:
# Create a new copy of the features summary dataframe to work with.
features_summary_cleaned_df = features_summary_original_df.copy()

In [ ]:
# Generated a list of unique values in the 'missing_or_unknown' column.
features_summary_cleaned_df['missing_or_unknown'].unique()


In [ ]:
# Capture the data type of the 'missing_or_unknown' column
features_summary_cleaned_df['missing_or_unknown'].dtype


In [ ]:
# Capture the data type of the 'missing_or_unknown' column's first value
type(features_summary_cleaned_df['missing_or_unknown'][0])


In [ ]:

# What this tells us is that allthough the values in the 'missing_or_unknown' column are stored as if they were lists 
# They are actually stored as a string data type. This means that if we were to attempt to iterate over the values in 
# each row of the 'missing_or_unknown' column, we would be iterating over each character in the string, rather than
# the individual values in the list. To fix this, we need to convert the string representation of the list into an actual  .

In [ ]:
from WGU_D499_P2_DCook.utils import convert_string_to_list, get_fs_string_conversions_failed

# Convert the 'missing_or_unknown' column from string representation of lists to actual lists.
features_summary_cleaned_df['missing_or_unknown'] = features_summary_cleaned_df.apply(convert_string_to_list, axis=1)

In [ ]:

# Get the list of features that failed the string conversion
fs_string_conversion_failed = get_fs_string_conversions_failed()

fs_string_conversion_failed   

In [ ]:
# Manually correcting these list items

features_summary_cleaned_df.at[57, 'missing_or_unknown'] = ['-1' , 'X']
features_summary_cleaned_df.at[58, 'missing_or_unknown'] = ['XX']
features_summary_cleaned_df.at[59, 'missing_or_unknown'] = ['-1' , 'XX']

In [ ]:
# This features dataframe will be used during the manual transformation and cleaning phase of this project
features_summary_manual_df = features_summary_cleaned_df.copy()

In [ ]:
# This features dataframe will be used during the automatic transformation and cleaning phase of this project
features_summary_auto_df = features_summary_cleaned_df.copy()

In [ ]:
# Beginning NAN Conversion

missing_unknown_identifier_df = features_summary_cleaned_df[['attribute', 'missing_or_unknown']].copy()

missing_unknown_identifier_df.reset_index(drop=True)

In [ ]:
from WGU_D499_P2_DCook.utils import convert_unknown_missing_to_nan


azdias_general_demographics_original_df, null_changes_dict = convert_unknown_missing_to_nan(azdias_general_demographics_original_df, missing_unknown_identifier_df, verbose=True)

In [ ]:
from WGU_D499_P2_DCook.utils import null_dict_to_dataframe

# Although we were able to see the output of the conversion above, it is not very easy to interpret the results.
# Let's convert the null changes dictionary to a DataFrame to make it easier to interpret the conversion results. 
null_conversion_results_dataframe = null_dict_to_dataframe(null_changes_dict)

In [ ]:
# display the null conversion results DataFrame
null_conversion_results_dataframe

In [ ]:
# We can even be extra careful and do a manual check on the null values in the azdias_general_demographics_original_df
# These values should all line up with what we have above. 

nan_null_check_df = azdias_general_demographics_original_df.copy()

for column in nan_null_check_df.columns:

    print(f"Column Name: {column}")
    print(f"Total Null/NP.Nan Values: {nan_null_check_df[column].isnull().sum()}")
    print()

#### Step 1.1.2: Assess Missing Data in Each Column

How much missing data is present in each column? There are a few columns that are outliers in terms of the proportion of values that are missing. You will want to use matplotlib's [`hist()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) function to visualize the distribution of missing value counts to find these columns. Identify and document these columns. While some of these columns might have justifications for keeping or re-encoding the data, for this project you should just remove them from the dataframe. (Feel free to make remarks about these outlier columns in the discussion, however!)

For the remaining features, are there any patterns in which columns have, or share, missing data?

In [ ]:
# Perform an assessment of how much missing data there is in each column of the
# dataset.

from WGU_D499_P2_DCook.eda import column_nan_eda_total_and_percentage

# Get the total and percentage of NaN values in each column of the dataset and store them in two separate series.
column_nan_eda_total_series, column_nan_eda_percentage_series = column_nan_eda_total_and_percentage(azdias_general_demographics_original_df)


In [ ]:
# Investigate patterns in the amount of missing data in each column.

from WGU_D499_P2_DCook.plots import plot_hist_column_nan_percentage, plot_bar_column_nan_percentage

# Plot the histogram of the percentage of NaN values in each column.
plot_hist_column_nan_percentage(column_nan_eda_percentage_series)

In [ ]:
# Plot the percentage of NaN values in each column as a bar chart.
plot_bar_column_nan_percentage(azdias_general_demographics_original_df)

In [ ]:
# Investigate patterns in the amount of missing data in each column.

from WGU_D499_P2_DCook.eda import eda_identify_outlier_columns, get_invalid_level_eda_identify_outlier_columns

# Identify columns with outliers based on the features summary and the general demographics data.
column_nan_eda_dataframe, outlier_column_name_list = eda_identify_outlier_columns(azdias_general_demographics_original_df, features_summary_manual_df, False, .2, ['Medium High','High', 'Very High'])


In [ ]:
# The "eda_identify_outlier_columns" function above returned a dataframe with the columns that have NaN values and their respective percentage of NaN values.
# This dataframe is used to identify the columns that have a high percentage of NaN values and are considered outliers.

# Priting the top thirty rows from the 'column_nan_eda_dataframe' with the head method. 
column_nan_eda_dataframe.head(30)



In [ ]:
from WGU_D499_P2_DCook.plots import plot_bar_eda_column_missing_severity

# Plot the bar chart of the columns with NaN values and their respective percentage of NaN values.
plot_bar_eda_column_missing_severity(column_nan_eda_dataframe)

In [ ]:

# The above "eda_identify_outlier_columns" method returns the outlier columns as well as the column_nan_eda_dataframe.
# While this method used a function to perform an contained/isolated get action to capture the outlier columns similar how getters/setters are used in O.O.P.
# So although we are not using this method to obtain the outlier columns, I wanted to keep the code here for reference.

# outlier_column_name_list = get_invalid_level_eda_identify_outlier_columns()

# Print the full list as a list
# outlier_column_name_list

# Print the list of outlier column names, more readable

for column_name in outlier_column_name_list:
    print(f"Outlier Column Name: {column_name}")



In [ ]:
# Remove the outlier columns from the dataset. (You'll perform other data engineering tasks such as re-encoding and imputation later.)

from WGU_D499_P2_DCook.transformations import remove_outlier_columns, update_features_summary_dataframe

# Here we are removing the outlier columns from the original dataframe using my 'remove_outlier_columns' function. 
# This function has seven total parameters, but we are only using the first three parameters here.
    # The first parameter is the original dataframe
    # The second parameter is the list of outlier column names
    # The third parameter is a boolean value to export the outlier columns to a local file.

# The function takes the original dataframe and the list of outlier column names, and removes those listed columns from the dataframe.

# Then the 'remove_outlier_columns' function will return a new dataframe with the outlier columns removed.

azdias_general_demographics_nan_col_df = remove_outlier_columns(azdias_general_demographics_original_df, outlier_column_name_list, True)



In [ ]:
from WGU_D499_P2_DCook.dataset import write_checkpoints

# Here I am writing the dataframe to a local file using my 'write_checkpoints' function to store a save state of the dataframe, a checkpoint if you will. 
write_checkpoints("interim_data", "azdias_general_demographics_nan_col_df", azdias_general_demographics_nan_col_df)

In [ ]:
# Here we updating the 'features_summary_manual_df' dataframe to remove the outlier columns that were removed from the original dataframe.
# My 'remove_outlier_columns' function has the ability to update the features summary dataframe as well, instead of doing this seperate function call. 
# However, I am having some issue with the function not working as expected, so I am doing this in a seperate function call.

features_summary_manual_df = update_features_summary_dataframe(features_summary_manual_df, outlier_column_name_list)

In [ ]:
# Visual top-level check that the features summary dataframe has been updated correctly.

features_summary_manual_df.info()

# Six items were removed so there should be now 79 items in the features summary dataframe, which there is.

In [ ]:
from WGU_D499_P2_DCook.dataset import write_checkpoints

# Here I am writing the dataframe to a local file using my 'write_checkpoints' function to store a save state of the dataframe, a checkpoint if you will. 
write_checkpoints("interim_data", "features_summary_manual_df", features_summary_manual_df)

#### Discussion 1.1.2: Assess Missing Data in Each Column

It's important that we start by defining what an 'Outlier' is.  In Mathematic and Statistics, an 'Outlier' is defined as a data point that significantly deviates from the other data points in a set. 

In our case, we would define a column as an 'outlier' if said column's contains a significantly larger amount of data that is considered missing compared to the rest of the columns in the dataset. 

This is not to say that missing data is not expected or at least not necessary a sign of a larger problem or bad/invalid data, our dataset is actually a perfect example of this. Earlier, you will recall when we begain by importing our dataset, we actually designated that any naturally missing values in the data were to be assigned as a Numpy NaN (np.NaN) value. 

From there, we then took it a step further and converted any values that were used to purposefully used to label data that was considered "missing", "unknown", "unspecified" into the same Numpy NaN value, effectively taking actual present data and turning them into Numpy NaN (missing) data. 

We did this because in our use-case we are attempting to train an Unsupervised Machine Learning model, and we cannot have missing values in our dataset. Furthermore, any areas were we are able to remove obvious signs of irrelevant data, such as data that holds no weight (like an entirely empty column), will make our training faster and result in more performant models. 

With that said, let's dive into how I determined what was considered to be an outlier and what columns did I find. 

In my exploratory data analysis, I being by calculating the sum of total number of missing values per column. Then I took that sum total and divided it by the total length of the column to create a percentage of total NaN values per column or the mean of the column's NaN. From here, I converted that number percentage to a score value by dividing it by 100, called the missing score and then created a baseline number by taking the average or mean of the all the missing scores. Using said baseline, I finished by assigning each column with a category that indicated the level of missing values, which I called this category the severity levels ( 'Very Low', 'Low', 'Medium Low', 'Medium', 'Medium High', 'High', 'Very High'). The level was determined by how far above or below the missing score was from the baseline mean that was just created. 

To make the entire process easier, I created a function to perform all the steps above and store them in a dataframe that can be reviewed or referenced if needed, which is shown above. Not only this, but my function also references the features summary and captures some extra details about each column, such as the level of data, and type of data it is, as well performing an export a list of the columns that were labeled as Outliers to a seperate file to be used later. 

For this project, I deciced that any column that was assigned with a severity level of 'Medium High', 'High', or 'Very High' would to be considered Outlier Columns. This resulted in a total of six columns that were labeled as outlier columns, or columns with abnormally high NaN percentages, they were:
1. TITEL_KZ
2. AGER_TYP
3. KK_KUNDENTYP
4. KBA05_BAUMAX
5. GEBURTSJAHR
6. ALTER_HH
 
Our exploratory data analysis shows that that average missing amount of data in each column is roughly 15.40%. These six columns above contain a range between 35% and almost 100% missing data. Three of the columns, TITEL_KZ, AGER_TYP, GEBURTSJAHR, are of the People information level. Two of the columns, ALTER_HH and KK_KUNDENTYP, are from the Household information level. The last column, KBA05_BAUMAX, comes from the RR3 mirco-cell level of information. 

Using the Data Dictionary, we are able to interpret that there is strong possibly of a correlation or at least link between the AGER_TYP, GEBURTSJAHR, and ALTER_HH features. Each of these features deal with the age of the consumer, AGER_TYPE attempts to define an elderly typology, the GEBURTSJAHR is defining the consumers year of birth, and the ALTER_HH wants to know the what date range the head of household's age falls in. 

For the TITEL_KZ and KBA05_BAUMAX columns, I believe their missing values follow a relatively easily prediction pattern, allow me to explain. In the TITEL_KZ column, we are indicating what level of advanced higher education degree the consumer has obtained, a Doctorate, Multiple Doctorates, Professor status, Professor Status with a Doctorate, or something similar. It is far more likely that the average consumer will not have a higher education degree. The KBA05_BAUMAX column wants to know how many and of what type of buildings exist in their Residental Rural 3 (RR3) microcell. If this information was obtained from the consumer directly/semi-indirectly, I would not expect this information to be known. 

Finally for the KK_KUNDENTYP column, this column inquires from the user what pattern of a consumer have they been over the last 12 months. I could see this information being left out if the question was worded poorly when obtained or even intentially left blank to prevent any potential consquence for providing feedback, i.e. If the information was gathered from a brand or service the customer like but happens to use sparsely, they may not want to review the limited use in case that has a determinal impact on the company-consumer relationship, etc. 

Regardless of the main reason, these columns have to go due to the amount of missing data, and so they were removed from the dataframe and exported to the archive_data directory for potential future research. 

After removing the outlier columns we have now reduced our dataset from a total of 85 features down to 79 features. If we look back, we can see that in the beginning we had a total of 62 columns out that contained missing data, but now with the six columns removed we are down to 56 columns that contain at least one missing item and 23 columns that contain 0. With that said, let's now turn our attention to the remaining columns that contained missing data and see if we are able to determine if there are any potential patterns. 

At initial glance, we are able to see a strong, potential pattern or correlation/relationship between at least several groups of features, based on the amount of missing data. 

The first of these groups contains seven features, which are: "PLZ8_ANTG1", "PLZ8_ANTG2", "PLZ8_ANTG3", "PLZ8_ANTG4", "PLZ8_HHZ", "PLZ8_BAUMAX", "PLZ8_GBZ". All seven of these features are from the same information level, macrocell_plz8, and have the exact same amount of issing data, 116,515 items or 13.0736%. 

The second of these groups contains six features, which are: "KBA05_ANTG1", "KBA05_ANTG2", "KBA05_ANTG3", "KBA05_ANTG4", "KBA05_GBZ", and "MOBI_REGIO". All six contain exactly 133,324 missing item or 14.9597% missing data. However, five of these features are from the same information level, microcell_rr3, and "MOBI_REGIO" comes from the region_rr1 information level and is related to movement patterns not building type/count like the others. We can also see that the the majority of these columns share a similar naming scheme as the ones from the first group. 

After this, we are still able to see groupings of data with the exact same amount of missing items and from the same information level. I would even consider it appropriate to for some of these features to infer that the data is significantly interconnected in some manner, most likely at the point of capture or its purpose/representation in the dataset. An example of the former would be with the "HEALTH_TYP", "SHOPPER_TYP", and "VERS_TYP" features, each have the same amount missing 111,196 or 12.1366%, each deals with the person level of data, and each asks a question about the user/consumer. For the latter, we have features like "LB_LEBENSPHASE_FEIN" and "LP_LEBENSPHASE_GROB" or "CAMEO_DEUG_2015", "CAMEO_DEU_2015", and "CAMEO_INTL_2015" or "GEBAEUDETYP" and "GEBAEUDETYP_RASTER" and several others. These features have either exactly the same or extremely little variance in the amount missing data and have very obvious connections based their naming and descriptions in the data dictionary, such as one feature represents the rough or high level overview of that feature and it's counterpart feature provides the fine or very granular level detail. 

These described groupings above make approximately roughly half of the remaining 56 columns. This initial inspection is not implying that these are an exact one to one value link (meaning the value that is missing in one group is a direct link to the other), this is just a high level review of missing data and interpretation of the numbers. If necessary, a more indepth analysis could be conducted into each column and the distribution of the missing values overall between all columns or even correlation studies on the features relationship. However, for now we are going to move forward with the data cleanup and model training and circle back around to the process improvement/quality control aspects another time. 


#### Step 1.1.3: Assess Missing Data in Each Row

Now, you'll perform a similar assessment for the rows of the dataset. How much data is missing in each row? As with the columns, you should see some groups of points that have a very different numbers of missing values. Divide the data into two subsets: one for data points that are above some threshold for missing values, and a second subset for points below that threshold.

In order to know what to do with the outlier rows, we should see if the distribution of data values on columns that are not missing data (or are missing very little data) are similar or different between the two groups. Select at least five of these columns and compare the distribution of values.
- You can use seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) function to create a bar chart of code frequencies and matplotlib's [`subplot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplot.html) function to put bar charts for the two subplots side by side.
- To reduce repeated code, you might want to write a function that can perform this comparison, taking as one of its arguments a column to be compared.

Depending on what you observe in your comparison, this will have implications on how you approach your conclusions later in the analysis. If the distributions of non-missing features look similar between the data with many missing values and the data with few or no missing values, then we could argue that simply dropping those points from the analysis won't present a major issue. On the other hand, if the data with many missing values looks very different from the data with few or no missing values, then we should make a note on those data as special. We'll revisit these data later on. **Either way, you should continue your analysis for now using just the subset of the data with few or no missing values.**

In [ ]:
# How much data is missing in each row of the dataset?

# The easiest way to do this would be to jut use the isnull() method with axis set to 1 for rows on the dataframe and then sum the results.

from WGU_D499_P2_DCook.eda import eda_identify_row_nan


eda_nan_row_dataframe, original_dataframe_len_ = eda_identify_row_nan(azdias_general_demographics_nan_col_df)


In [ ]:
# Graphing the results of the eda_identify_row_nan function
# This function will plot two histograms, one for the average missing value per row as a percentage of the whole column, and the other for the total number of missing values per row as a count.

# Version 1
# from WGU_D499_P2_DCook.plots import plot_hist_row_nan_eda
# plot_hist_row_nan_eda(eda_nan_row_dataframe)

#### #### #### #### #### #### #### #### 

# Version 2
# from WGU_D499_P2_DCook.plots import plot_hist_row_nan_eda_v2
# plot_hist_row_nan_eda_v2(eda_nan_row_dataframe)

#### #### #### #### #### #### #### #### 

# Final Version
from WGU_D499_P2_DCook.plots import plot_hist_row_nan_eda_side_by_side
from WGU_D499_P2_DCook.utils import display_describe_column_comparison_side_by_side

plot_hist_row_nan_eda_side_by_side(eda_nan_row_dataframe)

display_describe_column_comparison_side_by_side(eda_nan_row_dataframe, 'row_nan_total_mean', 'row_nan_total_sum', dc1_name = 'Proportion of Missing Values', dc2_name = 'Total Number of Missing Values')   

In [ ]:
# Write code to divide the data into two subsets based on the number of missing
# values in each row.

from WGU_D499_P2_DCook.eda import eda_row_nan_divide_by_threshold

# The 'eda_row_nan_divide_by_threshold' function will divide the dataframe into two subsets, low row NaN and high row NaN, based on a threshold value.

eda_low_row_nan_df, eda_high_row_nan_df, eda_row_nan_common_columns, eda_threshold_source, eda_threshold = eda_row_nan_divide_by_threshold(azdias_general_demographics_nan_col_df, max_loss_allowed=0.20, max_nan_per_row=0.10, method='kmeans', return_common_columns=True, allow_noise=False, noise_threshold=0.05, plot_graph=False)

In [ ]:
# Compare the distribution of values for at least five columns where there are
# no or few missing values, between the two subsets.

from WGU_D499_P2_DCook.plots import plot_countplout_compare_row_nan_distribution_per_column

# The 'plot_countplout_compare_row_nan_distribution_per_column' function will plot the distribution of values for a given column that is in both exists in both the eda_low_row_nan and eda_high_row_NaN dataframes.

for column in eda_row_nan_common_columns:
    plot_countplout_compare_row_nan_distribution_per_column(eda_low_row_nan_df, eda_high_row_nan_df, column)

In [ ]:
# Split the dataset into the two subsets

from WGU_D499_P2_DCook.transformations import row_nan_find_dynamic_nan_threshold, row_nan_divide_by_threshold

# The 'row_nan_find_dynamic_nan_threshold' function will find the optimal threshold value for dividing the dataset into two subsets based on the number of missing values in each row.
threshold = row_nan_find_dynamic_nan_threshold(azdias_general_demographics_nan_col_df, num_clusters = 3, num_init = 'auto', rand_stat = 5654)

# Output the determined threshold value
threshold

In [ ]:
from WGU_D499_P2_DCook.transformations import row_nan_find_dynamic_nan_threshold, row_nan_divide_by_threshold


# The 'row_nan_divide_by_threshold' function will divide the dataframe into two subsets, low row NaN and high row NaN, based on the threshold value determined by the 'row_nan_find_dynamic_nan_threshold' function.
azdias_general_demographics_low_row_nan_dataframe, azdias_general_demographics_high_row_nan_dataframe, row_nan_common_columns, threshold_source, threshold = row_nan_divide_by_threshold(azdias_general_demographics_nan_col_df, max_loss_allowed=0.20, max_nan_per_row=0.10, method='kmeans', return_common_columns=True, allow_noise=False, noise_threshold=0.05, plot_graph=True)



In [ ]:
from WGU_D499_P2_DCook.transformations import export_high_row_nan

# Export the high row NaN dataframe to a local file to be saved for later use.
export_high_row_nan(azdias_general_demographics_high_row_nan_dataframe, "archived_data", "azdias_general_demographics_High_Row_NaN")

In [ ]:

from WGU_D499_P2_DCook.eda import countplot_data_summary_multicol, calculate_proportion_diff, combine_proportion_diff_to_dataframe

#summary_results = countplot_data_summary_multicol(eda_low_row_nan_df, eda_high_row_nan_df, eda_row_nan_common_columns, normalize=False)
#summary_results = countplot_data_summary_multicol(eda_low_row_nan_df, eda_high_row_nan_df, eda_row_nan_common_columns, normalize=False)

summary_results = countplot_data_summary_multicol(eda_low_row_nan_df, eda_high_row_nan_df, eda_row_nan_common_columns)

# Or loop through all:
#for column, dataframe in summary_results.items():
    #print(f"\n=== {column} ===")
    #print(dataframe)


differences_results = calculate_proportion_diff(summary_results)

differences_dataframe = combine_proportion_diff_to_dataframe(differences_results)
differences_dataframe




In [ ]:
from WGU_D499_P2_DCook.plots import plot_barplot_compare_row_nan_dist_per_column_percentage

for column in eda_row_nan_common_columns:
    plot_barplot_compare_row_nan_dist_per_column_percentage(eda_low_row_nan_df, eda_high_row_nan_df, column)

In [ ]:
# Creating a new dataframe from a copy of the low row NaN dataframe to be used moving forward. 
azdias_general_demographics_col_row_nan_dataframe = azdias_general_demographics_low_row_nan_dataframe.copy()

In [ ]:
from WGU_D499_P2_DCook.utils import check_and_remove_row_nan_columns

# Checking to see if any of the row NaN functions columns there were generated still exist in the dataframe, 
# columns, such as:  'row_nan_total_mean' and the 'row_nan_category' columns.
# If they do exist, then we will remove them from the dataframe.
azdias_general_demographics_col_row_nan_dataframe = check_and_remove_row_nan_columns(azdias_general_demographics_col_row_nan_dataframe)

In [ ]:
from WGU_D499_P2_DCook.dataset import write_checkpoints

# Here I am writing the dataframe to a local file using my 'write_checkpoints' function to store a save state of the dataframe, a checkpoint if you will. 
write_checkpoints("interim_data", "azdias_general_demographics_col_row_nan_dataframe", azdias_general_demographics_col_row_nan_dataframe)

In [ ]:

# Create a new copy of the dataframe to be used moving forward.
azdias_general_demographics_main_df = azdias_general_demographics_col_row_nan_dataframe.copy()

#### Discussion 1.1.3: Assess Missing Data in Each Row

(Double-click this cell and replace this text with your own text, reporting your observations regarding missing data in rows. Are the data with lots of missing values are qualitatively different from data with few or no missing values?)

In the previous section, we identified and removed the features that contained the majority of the missing data, the outlier columns. We also noted in the previous section that roughly two-thirds of  our updated dataset still contained missing values interdispersed throughout the features. As we stated before, these missing values are not necessarily an issue since we have translated values that were purposeful labeled as "missing", "unknown", or "unspecified" into the same value that naturally missing values would be, Numpy NaN. Let's take a closer look into how these missing values are distributed throughout each of the rows throughout our dataset. 

We started by taking our updated dataset and running some simple exploratory data analysis on it to get some rough figures. Using an EDA function I created, we will take a copy of our dataset and create two new columns and append them to the dataset. The first column will store the total number of features that contain a missing value in each row using the sum method, and we will save it into a new column called 'row_nan_total_sum'. The second column will be called 'row_nan_total_mean' and it will store the average number of missing values per row created using the mean method. This function then will print out some descriptive statistics using the describe method and return the new row nan eda dataframe and a value containing the length of the original dataframe calculated using the len function.  

It is here that we can will pass the returned EDA dataframe into our plot functions, specifically the 'plot_hist_row_nan_eda_side_by_side'. This function creates two histograms side by side by plotting each of the previously created row nan columns. The first of these histograms shows us the average percentage of missing values from each row binned into groups. Here we can see that the majority of our columns have less than 10% missing data. The second column shows the total count of missing values per row as a summed total that has been binned as well. This column again validates that the majority of our missing data is less than 10 items per row. The second function we run displays the descriptive statistics generated from the describe method in a side by side dataframe as well. Our visual analysis is further backed up by these stats. We can see that on average missing per row in the proportional eda is roughly 7% missing data from each row with a deviation of 16% in this mean. Our upper quartile of missing data is less than 4%. The average missing in our count totals per row is roughly 6 items, with a deviation of roughly 13 items. The max value see was 49 items, which is 62% of the row, and our upper quartiles have 3 items missing per row. 

This gives us some good idea of how our missing data is distributed. Howevver, let's take it a step further and work towards the isolation and removal of rows that would be considered subpar from a quality stand-point. To do this, we are going to start by dividing our dataset into two parts, one for the rows with a low percentage of missing data and the other for rows for rows that we have determined have a high amount of missing data. However, how should we divided our dataset? Should we divide using a hard-coded maximum value? Should we dynamically calculate the ideal amount? What about quanity? How much data loss from removal is acceptable? These are all really good questions to have when approaching a problem such as this. 

There really isn't a one size fits all solution and we have to be adaptable to resolve these. This is why I decided to take the more difficult approach and design a solution that would be dynamic in nature but also have some fallbacks or fail-safes in place. My solution attempts to dyanamically locate and assign a threshold value to divide my dataset into either low or high. It actually does it by utilizing a technique that we will cover more so later in the project, Clustering. We perform a KMeans clustering analysis on our dataframe. We do this by first converting our dataset into a series of the average percentage of nan values per row. We then reshape this data and pass it into our KMeans object. Once we have fit our data to the KMeans Object, we will extract the center values from our cluster object, and then sort and flatten them into a a list which we will obtain the average value of our centers. This value is what we will set as our optimal threshold value. Now that we have located what we believe is the optimal threshold value to safely divide our dataset into two, we then perform an comparision between the mean value of missing values per row in our dataset against our opitmal threshold value. This generates us a series of true and false values for which we will obtain the porition or average of our dataset that is above our threshold value. This average is then assigned to the 'actual_loss' variable. Why do we do this, this is the first step to one of my safe guards, the actual loss value represents how much of our dataset will be lost if we decided to split our dataset using that threshold value. How is this important you may ask? By knowing how much of our data will be removed by this threshold value, we are able to setup fallbacks to allow us to prevent against shrinking out dataset and affecting the overal quality of our data. It's extremely important to have good quality data, but having too small of a dataset can create issues of its own. So its a balancing act on how much of our data we should keep verus how good the quality of our data is. In my solution, I decided that I never wanted to lose more than 20% of my total data. So how is this used? Well if the threshold value we determiend using our Kmeans clustering analysis was found to create an actual loss of data greater than 20%, then what we will do is fall back to a hard coded threshold value we have set, which in our case is 10% maximum allowed missing data. What about if said hard coded threshold causes greater than 20% loss of data? In my current design, I have built my function to prioritize good quality data in quanitity size. However, if we end up in a fall back scenario due to maximium loss of data with our dynamic threshold value, then our priority scope will shift from a quanity focus to almost purely quality focused. Meaning we will filter out all rows that have greater than 10% missing data. 

So at this point, we have determined a threshold to divide our dataset, so let's do that using our obtained threshold value. To do this, my function then makes a copy of the dataset and creates a new column to hold the row nan mean value. We create a second column to hold a category value based on that rows mean value, if the value is above our threshold value, then the row is labeled as 'high' and if the value is lower than it is labeled as 'low'. Then we split the dataset into two subset based on the row's label. 

Now that we have split the two datasets, let's compare them to see if they have any commonality or patterns to the distribution that we can see. To do this, we will create a new list containing only columns that exist in both datasets for a fair review. We do this by appending any column name that is found to exist in both dataframes.  It is here that you will notice that I have added a noise feature to my comparision. In my function, I added the ability to either accept only columns that have no missing values or the ability to define an acceptable percentage level of missing values per row in each column. If the values are above the accpetable level of noise then they are not included in the column comparision analysis, if the noise functionality is enabled. From there, we will save the list of column names for future reference/usage.

So now that we've split our dataframe into two subsets and obtained a list of columns that exists in both our low row nan eda dataframe and our high row nan dataframe, we can now graph and compare our results to see the distribution of values between the two dataset. We do this iterating through a list of using the list of common columns to we just generated and both our dataframes ploting both graphs side by side using the countplot function. 






### Step 1.2: Select and Re-Encode Features

Checking for missing data isn't the only way in which you can prepare a dataset for analysis. Since the unsupervised learning techniques to be used will only work on data that is encoded numerically, you need to make a few encoding changes or additional assumptions to be able to make progress. In addition, while almost all of the values in the dataset are encoded using numbers, not all of them represent numeric values. Check the third column of the feature summary (`feat_info`) for a summary of types of measurement.
- For numeric and interval data, these features can be kept without changes.
- Most of the variables in the dataset are ordinal in nature. While ordinal values may technically be non-linear in spacing, make the simplifying assumption that the ordinal variables can be treated as being interval in nature (that is, kept without any changes).
- Special handling may be necessary for the remaining two variable types: categorical, and 'mixed'.

In the first two parts of this sub-step, you will perform an investigation of the categorical and mixed-type features and make a decision on each of them, whether you will keep, drop, or re-encode each. Then, in the last part, you will create a new data frame with only the selected and engineered columns.

Data wrangling is often the trickiest part of the data analysis process, and there's a lot of it to be done here. But stick with it: once you're done with this step, you'll be ready to get to the machine learning parts of the project!

In [ ]:
# How many features are there of each data type?

from WGU_D499_P2_DCook.utils import get_feature_types_and_count

feature_counts, interval_columns, numeric_columns, ordinal_columns, categorical_columns, mixed_columns = get_feature_types_and_count(features_summary_manual_df)

feature_counts

#### Step 1.2.1: Re-Encode Categorical Features

For categorical data, you would ordinarily need to encode the levels as dummy variables. Depending on the number of categories, perform one of the following:
- For binary (two-level) categoricals that take numeric values, you can keep them without needing to do anything.
- There is one binary variable that takes on non-numeric values. For this one, you need to re-encode the values as numbers or create a dummy variable.
- For multi-level categoricals (three or more values), you can choose to encode the values using multiple dummy variables (e.g. via [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)), or (to keep things straightforward) just drop them from the analysis. As always, document your choices in the Discussion section.

In [ ]:
# Assess categorical variables: which are binary, which are multi-level, and
# which one needs to be re-encoded?

from WGU_D499_P2_DCook.eda import eda_column_identify_binary_multilevel_category

binary_set = set()
multi_set = set()
error_set = set()

binary_set, multi_set, error_set = eda_column_identify_binary_multilevel_category(azdias_general_demographics_main_df, categorical_columns, binary_set, multi_set, error_set)



In [ ]:

binary_columns_first_pass_list = list(binary_set)
multilevel_columns_first_pass_list = list(multi_set)
error_columns_first_pass_list = list(error_set)

In [ ]:

binary_columns_first_pass_list


In [ ]:

print(len(binary_columns_first_pass_list))
print(len(set(binary_columns_first_pass_list)))



In [ ]:
from WGU_D499_P2_DCook.utils import column_dtype_check

column_dtype_check(azdias_general_demographics_main_df, binary_columns_first_pass_list)

In [ ]:
# Re-encode categorical variable(s) to be kept in the analysis.

from WGU_D499_P2_DCook.transformations import apply_if_column_exists
from WGU_D499_P2_DCook.features import column_correction_ost_west_kz 


azdias_general_demographics_main_df =  apply_if_column_exists(
    azdias_general_demographics_main_df,
    'OST_WEST_KZ',
    column_correction_ost_west_kz,
    remove_if_missing=True,
    remove_from_lists=[binary_columns_first_pass_list, multilevel_columns_first_pass_list],
    apply_per_column=True, 
    keep_original = False
)


In [ ]:
from WGU_D499_P2_DCook.transformations import apply_if_column_exists
from WGU_D499_P2_DCook.features import column_correction_vers_type 


azdias_general_demographics_main_df =  apply_if_column_exists(
    azdias_general_demographics_main_df,
    'VERS_TYP',
    column_correction_vers_type,
    remove_if_missing=True,
    remove_from_lists=[binary_columns_first_pass_list, multilevel_columns_first_pass_list],
    apply_per_column=True
)

In [ ]:
from WGU_D499_P2_DCook.transformations import apply_if_column_exists
from WGU_D499_P2_DCook.features import column_correction_anrede_kz 


azdias_general_demographics_main_df =  apply_if_column_exists(
    azdias_general_demographics_main_df,
    'ANREDE_KZ',
    column_correction_anrede_kz,
    remove_if_missing=True,
    remove_from_lists=[binary_columns_first_pass_list, multilevel_columns_first_pass_list],
    apply_per_column=True
)

In [ ]:
multilevel_columns_first_pass_list

In [ ]:
from WGU_D499_P2_DCook.transformations import apply_if_column_exists
from WGU_D499_P2_DCook.features import column_correction_cameo_deu_2015 

azdias_general_demographics_main_df =  apply_if_column_exists(
    azdias_general_demographics_main_df,
    'CAMEO_DEU_2015',
    column_correction_cameo_deu_2015,
    remove_if_missing=True,
    remove_from_lists=[binary_columns_first_pass_list, multilevel_columns_first_pass_list],
    apply_per_column=True
)

In [ ]:

convert_familie_int_to_label = {1: 'single', 2: 'couple', 3: 'single_parent', 4: 'family', 5: 'multihousehold'}

convert_status_int_to_label = {1: 'low_income', 2: 'avg_income', 3: 'independant', 4: 'houseowner', 5: 'top_earner'}




In [ ]:

original_column_names=['LP_FAMILIE_GROB','LP_STATUS_GROB']

map_dictionary_list=[[convert_familie_int_to_label], [convert_status_int_to_label]]
                     
new_columns_list=[]

In [ ]:
from WGU_D499_P2_DCook.transformations import apply_if_column_exists
from WGU_D499_P2_DCook.features import feature_mapping 


azdias_general_demographics_main_df = apply_if_column_exists(
    azdias_general_demographics_main_df,
    original_column_names,
    feature_mapping,
    True,
    [binary_columns_first_pass_list, multilevel_columns_first_pass_list],
    False,
    original_column_names, map_dictionary_list, new_columns_list, True, False, False, True
)


In [ ]:
multilevel_columns_first_pass_list.remove("LP_STATUS_FEIN")
multilevel_columns_first_pass_list.remove("LP_FAMILIE_FEIN")


In [ ]:
one_hot_encode_column_list = multilevel_columns_first_pass_list

In [ ]:
one_hot_encode_column_list

In [ ]:
from WGU_D499_P2_DCook.features import column_encode_multi_category_to_single_onehot


for column in one_hot_encode_column_list:
    azdias_general_demographics_main_df = column_encode_multi_category_to_single_onehot(azdias_general_demographics_main_df, column, drop_first_flag = False, drop_original = True)



In [ ]:
from WGU_D499_P2_DCook.utils import convert_column_names_to_uppercase


azdias_general_demographics_main_df = convert_column_names_to_uppercase(azdias_general_demographics_main_df)



In [ ]:
azdias_general_demographics_main_df.columns

#### Discussion 1.2.1: Re-Encode Categorical Features

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding categorical features. Which ones did you keep, which did you drop, and what engineering steps did you perform?)

#### Step 1.2.2: Engineer Mixed-Type Features

There are a handful of features that are marked as "mixed" in the feature summary that require special treatment in order to be included in the analysis. There are two in particular that deserve attention; the handling of the rest are up to your own choices:
- "PRAEGENDE_JUGENDJAHRE" combines information on three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west). While there aren't enough levels to disentangle east from west, you should create two new variables to capture the other two dimensions: an interval-type variable for decade, and a binary variable for movement.
- "CAMEO_INTL_2015" combines information on two axes: wealth and life stage. Break up the two-digit codes by their 'tens'-place and 'ones'-place digits into two new ordinal variables (which, for the purposes of this project, is equivalent to just treating them as their raw numeric values).
- If you decide to keep or engineer new features around the other mixed-type features, make sure you note your steps in the Discussion section.

Be sure to check `Data_Dictionary.md` for the details needed to finish these tasks.

In [ ]:
# Investigate "PRAEGENDE_JUGENDJAHRE" and engineer two new variables.


# PRAEGENDE_JUGENDJAHRE --> three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west)
# two new variables :: an interval-type variable for decade, and a binary variable for movement.
#     Decades: 40s, 50s, 60s, 70s, 80s, 90s
#     Movement: Mainstream = 0, Avantgarde = 1
#     Nation: Can either do One-Hot Encoding -- Possibly more manual then use of the function -- where E+W equals 1 and 1 for both east and west. 
#          Or.. 
#             I can use multihot encoding (MultiLabelBinarizer from sklearn.preprocessing) labeling the nation as [East], [West], ['East', 'West']
#             Then run that through MultiLabelBinarizer() then encode it into two new columns for east and west. The multi-hot encoding allows for multiple 1s in a row

### 1.18. PRAEGENDE_JUGENDJAHRE
# Data dictionary
# Dominating movement of person's youth (avantgarde vs. mainstream; east vs. west)
# - -1: unknown
# -  0: unknown
# -  1: 40s - war years (Mainstream, E+W)
# -  2: 40s - reconstruction years (Avantgarde, E+W)
# -  3: 50s - economic miracle (Mainstream, E+W)
# -  4: 50s - milk bar / Individualisation (Avantgarde, E+W)
# -  5: 60s - economic miracle (Mainstream, E+W)
# -  6: 60s - generation 68 / student protestors (Avantgarde, W)
# -  7: 60s - opponents to the building of the Wall (Avantgarde, E)
# -  8: 70s - family orientation (Mainstream, E+W)
# -  9: 70s - peace movement (Avantgarde, E+W)
# - 10: 80s - Generation Golf (Mainstream, W)
# - 11: 80s - ecological awareness (Avantgarde, W)
# - 12: 80s - FDJ / communist party youth organisation (Mainstream, E)
# - 13: 80s - Swords into ploughshares (Avantgarde, E)
# - 14: 90s - digital media kids (Mainstream, E+W)
# - 15: 90s - ecological awareness (Avantgarde, E+W)

# Decades:
# 40s - 1, 2
# 50s - 3, 4
# 60s - 5, 6, 7
# 70s - 8, 9, 
# 80s - 10, 11, 12, 13
# 90s - 14, 15

# Movement:
# Mainstream: 1, 3, 5, 8, 10, 12, 14
# Avantgrade: 2, 4, 6, 7, 9, 11, 13, 15

# East & West
# East: 7, 12, 13
# West: 6, 10, 11, 
# East & West: 1, 2, 3, 4, 5, 8, 9, 14, 15

from WGU_D499_P2_DCook.transformations import apply_if_column_exists
from WGU_D499_P2_DCook.features import column_reengineering_pragende_jugendjahre

azdias_general_demographics_main_df =  apply_if_column_exists(
    azdias_general_demographics_main_df,
    'PRAEGENDE_JUGENDJAHRE',
    column_reengineering_pragende_jugendjahre,
    one_hot_encode=True, keep_original=False,
    remove_if_missing=False,
    remove_from_lists=None,
    apply_per_column=True
)




In [ ]:
# Investigate "CAMEO_INTL_2015" and engineer two new variables.

# Investigate "CAMEO_INTL_2015" and engineer two new variables.
# CAMEO_INTL_2015 --> two axes: wealth and life stage
# Convert two digit, signle code to 2 single value codes. 
#     Tens Place = Wealth Status
#         1 = Wealthy, 
#         2 = Prosperous, 
#         3 = Comfortable, 
#         4 = Less Affluent, 
#         5 = Pooer
#     Ones Place = Life Stage Typology (Struture/Makeup/etc)
#         1 = PreFamily Couples/Singles, 
#         2 = Young Couples With Children, 
#         3 = Families With School Age Children, 
#         4 = Older Familys & Mature Couples, 
#         5 = Elders In Retirement


from WGU_D499_P2_DCook.transformations import apply_if_column_exists
from WGU_D499_P2_DCook.features import column_reengineering_cameo_intl_2015

azdias_general_demographics_main_df =  apply_if_column_exists(
    azdias_general_demographics_main_df,
    'CAMEO_INTL_2015',
    column_reengineering_cameo_intl_2015,
    one_hot_encode=True, keep_original=False,
    remove_if_missing=False,
    remove_from_lists=None,
    apply_per_column=True
)




#### Discussion 1.2.2: Engineer Mixed-Type Features

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding mixed-value features. Which ones did you keep, which did you drop, and what engineering steps did you perform?)

#### Step 1.2.3: Complete Feature Selection

In order to finish this step up, you need to make sure that your data frame now only has the columns that you want to keep. To summarize, the dataframe should consist of the following:
- All numeric, interval, and ordinal type columns from the original dataset.
- Binary categorical features (all numerically-encoded).
- Engineered features from other multi-level categorical features and mixed features.

Make sure that for any new columns that you have engineered, that you've excluded the original columns from the final dataset. Otherwise, their values will interfere with the analysis later on the project. For example, you should not keep "PRAEGENDE_JUGENDJAHRE", since its values won't be useful for the algorithm: only the values derived from it in the engineered features you created should be retained. As a reminder, your data should only be from **the subset with few or no missing values**.

In [ ]:
# If there are other re-engineering tasks you need to perform, make sure you
# take care of them here. (Dealing with missing data will come in step 2.1.)

from WGU_D499_P2_DCook.transformations import apply_if_column_exists 
from WGU_D499_P2_DCook.features import column_reengineering_lp_lebenshase, column_reengineering_wohnlage, column_reengineering_plz8_family_business_building_flag


azdias_general_demographics_main_df =  apply_if_column_exists(
    azdias_general_demographics_main_df,
    'LP_LEBENSPHASE_FEIN',
    column_reengineering_lp_lebenshase,
    one_hot_encode = True, keep_original = False,
    remove_if_missing=False,
    remove_from_lists=None,
    apply_per_column=True
)




azdias_general_demographics_main_df =  apply_if_column_exists(
    azdias_general_demographics_main_df,
    'WOHNLAGE',
    column_reengineering_wohnlage,
    keep_original = False,
    remove_if_missing=False,
    remove_from_lists=None,
    apply_per_column=True
)


azdias_general_demographics_main_df =  apply_if_column_exists(
    azdias_general_demographics_main_df,
    'PLZ8_BAUMAX',
    column_reengineering_plz8_family_business_building_flag,
    keep_original = False,
    remove_if_missing=False,
    remove_from_lists=None,
    apply_per_column=True
)

In [ ]:
# Do whatever you need to in order to ensure that the dataframe only contains
# the columns that should be passed to the algorithm functions.

#azdias_general_demographics = azdias_general_demographics.drop(columns='LP_LEBENSPHASE_GROB')
#azdias_general_demographics = azdias_general_demographics.drop(columns='LP_FAMILIE_FEIN')
#azdias_general_demographics = azdias_general_demographics.drop(columns='LP_STATUS_FEIN')

# Drop the following mixed columns
azdias_general_demographics_main_df.drop(columns=['LP_STATUS_FEIN', 'LP_FAMILIE_FEIN', 'LP_LEBENSPHASE_GROB'], inplace=True)

In [ ]:
from WGU_D499_P2_DCook.utils import convert_column_names_to_uppercase

azdias_general_demographics_main_df = convert_column_names_to_uppercase(azdias_general_demographics_main_df)

In [ ]:
azdias_general_demographics_cleaned_prescaled_main_df = azdias_general_demographics_main_df.copy()

In [ ]:
from WGU_D499_P2_DCook.utils import get_info_to_dataframe

get_info_to_dataframe(azdias_general_demographics_cleaned_prescaled_main_df)

In [ ]:

from WGU_D499_P2_DCook.dataset import write_checkpoints

# Here I am writing the dataframe to a local file using my 'write_checkpoints' function to store a save state of the dataframe, a checkpoint if you will. 
write_checkpoints("interim_data", "azdias_general_demographics_cleaned_prescaled_main_df", azdias_general_demographics_cleaned_prescaled_main_df)

### Step 1.3: Create a Cleaning Function

Even though you've finished cleaning up the general population demographics data, it's important to look ahead to the future and realize that you'll need to perform the same cleaning steps on the customer demographics data. In this substep, complete the function below to execute the main feature selection, encoding, and re-engineering steps you performed above. Then, when it comes to looking at the customer data in Step 3, you can just run this function on that DataFrame to get the trimmed dataset in a single step.

In [ ]:
def clean_data(cd_dataframe_original, cd_features_dataframe, ft_ops, FeatureTransformer):
    """
    Cleans the input dataframe by performing various data cleaning steps such as
    converting missing value codes into NaNs, removing selected columns and rows,
    selecting, re-encoding, and engineering column values.
    Parameters:
    - dataframe: pd.DataFrame
        The input dataframe to be cleaned.  
    - features_dataframe: pd.DataFrame
        The features summary dataframe containing information about the columns.
    - ft_ops: list
        A list of operations to be performed on the dataframe through the FeatureTransformer class.
    - FeatureTransformer: class
        A class that contains methods for transforming the dataframe based on the specified operations.

    Returns:    
    - pd.DataFrame
        The cleaned dataframe after performing all the specified operations.


    """
    
    # Put in code here to execute all main cleaning steps:
    # convert missing value codes into NaNs, ...

    from WGU_D499_P2_DCook.dataset import write_checkpoints
    from WGU_D499_P2_DCook.utils import convert_unknown_missing_to_nan, check_and_remove_row_nan_columns, get_feature_types_and_count, convert_column_names_to_uppercase
    from WGU_D499_P2_DCook.eda import eda_identify_outlier_columns, get_invalid_level_eda_identify_outlier_columns, eda_column_identify_binary_multilevel_category
    from WGU_D499_P2_DCook.transformations import remove_outlier_columns, update_features_summary_dataframe, row_nan_divide_by_threshold, export_high_row_nan
    from WGU_D499_P2_DCook.features import FeatureTransformer, column_correction_cameo_deu_2015, column_reengineering_cameo_intl_2015, column_encode_multi_category_to_single_onehot


    # Convert features summary dataframe into missing or unknown identifier dataframe.

    cd_missing_unknown_identifier_df = cd_features_dataframe[['attribute', 'missing_or_unknown']].copy()
    cd_missing_unknown_identifier_df.reset_index(drop=True)

    # Convert unknown and missing values in the main dataframe to NaN.
    cd_dataframe_original, null_dictionary = convert_unknown_missing_to_nan(cd_dataframe_original, cd_missing_unknown_identifier_df, verbose=True)
    
    # Create a copy of the original dataframe for as a checkpoint.
    write_checkpoints("archived_data", "cd_dataframe_original", cd_dataframe_original)

    # Identify columns with outliers 
    cd_column_nan_eda_dataframe, cd_outlier_column_name_list = eda_identify_outlier_columns(cd_dataframe_original, cd_features_dataframe, False, .2, ['Medium High','High', 'Very High'])

    # Print the outlier column names
    print("Outlier Columns Identified:")
    for column in cd_outlier_column_name_list:
        print(f" - {column}")

    # Print Invalid Level Columns Identified
    cd_invalid_level = get_invalid_level_eda_identify_outlier_columns()

    if cd_invalid_level:
        print("Invalid Level Columns Identified:")
        for column in cd_invalid_level:
            print(f" - {column}")
            
    # Create a copy of the column eda dataframe for as a checkpoint.
    write_checkpoints("archived_data", "cd_eda_column_nan_dataframe", cd_column_nan_eda_dataframe)

    # Remove outlier columns from the main dataframe and updated features summary dataframe.
    cd_dataframe_col_outlier_removed = remove_outlier_columns(cd_dataframe_original, cd_outlier_column_name_list, export_outliers_to_file = True, export_file_path = "archived_data", export_file_name = "cd_outlier_columns", update_features_summary_dataframe = False, features_summary_dataframe_object = cd_features_dataframe)
    print(cd_dataframe_col_outlier_removed.info())
    print(cd_dataframe_col_outlier_removed.shape)

    # Issues with Update_features_summary call from Remove_outlier Columns function. 
    # Will need to troubleshoot. Calling manually for now. 
    cd_features_dataframe_col_outliers_removed = update_features_summary_dataframe(cd_features_dataframe, cd_outlier_column_name_list)
    print(cd_features_dataframe_col_outliers_removed.info())
    print(cd_features_dataframe_col_outliers_removed.shape)

    # Create a copy of the column eda dataframe for as a checkpoint.
    write_checkpoints("archived_data", "cd_dataframe_outliers_removed", cd_dataframe_col_outlier_removed)
    write_checkpoints("archived_data", "cd_features_dataframe_col_outliers_removed", cd_features_dataframe_col_outliers_removed)

    # Identify and and split dataframe into rows with low and high NaN counts based on a dynamic threshold.
    cd_low_row_nan_dataframe, cd_high_row_nan_dataframe, cd_row_nan_common_columns, cd_threashold_source, cd_threshold = row_nan_divide_by_threshold(cd_dataframe_col_outlier_removed, max_loss_allowed=0.20, max_nan_per_row=0.10, method='kmeans', return_common_columns=True, allow_noise=False, noise_threshold=0.05, plot_graph=True) 
    
    print(f"Threshold Source: {cd_threashold_source}")
    print(f"Threshold Value: {cd_threshold}")
    print(f"Common Columns: {cd_row_nan_common_columns}")

    
    # Export the high row NaN dataframe to a file.
    export_high_row_nan(cd_high_row_nan_dataframe, "archived_data", "cd_high_row_nan_dataframe")

    # Save the low row NaN dataframe to a new variable for further processing.
    cd_dataframe_col_row_cleaned = cd_low_row_nan_dataframe.copy()
    print(cd_dataframe_col_row_cleaned.info())
    print(cd_dataframe_col_row_cleaned.shape)

    # Check and remove columns with NaN values in the low row NaN dataframe.
    cd_dataframe_col_row_cleaned = check_and_remove_row_nan_columns(cd_dataframe_col_row_cleaned)

    # Create a copy of the cleaned dataframe for as a checkpoint before any further transformations.
    write_checkpoints("archived_data", "cd_dataframe_col_row_nans_cleaned", cd_dataframe_col_row_cleaned)

    # Get the feature types and counts from the features summary dataframe.
    cd_feature_counts, cd_interval_columns, cd_numeric_columns, cd_ordinal_columns, cd_categorical_columns, cd_mixed_columns = get_feature_types_and_count(cd_features_dataframe_col_outliers_removed)
    
    cd_binary_set = set()
    cd_multi_set = set()
    cd_error_set = set()

    eda_column_identify_binary_multilevel_category(cd_dataframe_col_row_cleaned, cd_categorical_columns, cd_binary_set, cd_multi_set, cd_error_set)

    # Convert the sets to lists for further processing.
    cd_binary_columns_first_pass_list = list(cd_binary_set)
    cd_multilevel_columns_first_pass_list = list(cd_multi_set)
    cd_error_columns_first_pass_list = list(cd_error_set)

    if not cd_error_columns_first_pass_list:
        print("Error Columns Identified:")
        for column in cd_error_columns_first_pass_list: 
            print(f" - {column}")



    # select, re-encode, and engineer column values.
    ft = FeatureTransformer(operations=ft_ops)
    
    # Transformating the dataframe using the FeatureTransformer class.
    cd_dataframe_transformed = ft.transform_safe(cd_dataframe_col_row_cleaned) 

    print("Skipped columns:", ft.skipped_columns)

    # dropping the following columns
    cd_dataframe_transformed = cd_dataframe_transformed.drop(columns=[
        'LP_LEBENSPHASE_GROB', 'LP_FAMILIE_FEIN', 'LP_STATUS_FEIN'
    ], errors='ignore')


    # Performing column re-engineering on cameo_intl_2015.
    cd_dataframe_transformed = column_reengineering_cameo_intl_2015(cd_dataframe_transformed, one_hot_encode=True, keep_original = False)

    # Performing column re-engineering on cameo_deu_2015.
    cd_dataframe_transformed = column_correction_cameo_deu_2015(cd_dataframe_transformed)


    # Re-encode categorical variable(s) to be kept in the analysis.
    cd_multilevel_columns_first_pass_list.remove("LP_STATUS_FEIN")

    cd_multilevel_columns_first_pass_list.remove("LP_FAMILIE_FEIN")

    # Saving the multilevel category column list into one-hot encoded columns list.
    cd_one_hot_encode_column_list = cd_multilevel_columns_first_pass_list
    print(cd_one_hot_encode_column_list)

    # Saving dataframe into a new dataframe prior to one-hot encoding.
    cd_dataframe_transformed_pre_encoded = cd_dataframe_transformed.copy()

    # Writing a checkpoint of the pre-encoded dataframc
    write_checkpoints("archived_data", "cd_dataframe_tc_encoded", cd_dataframe_transformed_pre_encoded)

    # One-hot encoding the multi-category columns.
    for column in cd_one_hot_encode_column_list:
        print("starting one hot encoding")
        print(column)
        cd_dataframe_transformed_pre_encoded = column_encode_multi_category_to_single_onehot(cd_dataframe_transformed_pre_encoded, column, drop_first_flag = False, drop_original = True)
        print(cd_dataframe_transformed_pre_encoded.shape)

    cd_dataframe_transformed_encoded = cd_dataframe_transformed_pre_encoded.copy()

    write_checkpoints("archived_data", "cd_dataframe_transformed_encoded", cd_dataframe_transformed_encoded)

    # Convert column names to uppercase.
    cd_dataframe_transformed_encoded = convert_column_names_to_uppercase(cd_dataframe_transformed_encoded)

    # Pre-fill NaN values with 0.
    cd_dataframe_transformed_encoded['PLZ8_BAUMAX_FAMILY_BUILDING'] = cd_dataframe_transformed_encoded['PLZ8_BAUMAX_FAMILY_BUILDING'].fillna(0).astype(float)
    cd_dataframe_transformed_encoded['PLZ8_BAUMAX_BUSINESS_BUILDING'] = cd_dataframe_transformed_encoded['PLZ8_BAUMAX_BUSINESS_BUILDING'].fillna(0).astype(float)

    cd_dataframe_transformed_encoded['WOHNLAGE_NEIGHBOURHOOD'] = cd_dataframe_transformed_encoded['WOHNLAGE_NEIGHBOURHOOD'].fillna(0).astype(float)
    cd_dataframe_transformed_encoded['WOHNLAGE_RURAL'] = cd_dataframe_transformed_encoded['WOHNLAGE_RURAL'].fillna(0).astype(float)


    # Create a copy of the cleaned dataframe for as a checkpoint before any further transformations.
    write_checkpoints("interim_data", "cd_dataframe_transformed_encoded", cd_dataframe_transformed_encoded)


    # Return the cleaned dataframe.
    
    # Debugging
    #return cd_feature_counts, cd_interval_columns, cd_numeric_columns, cd_ordinal_columns, cd_categorical_columns, cd_mixed_columns, cd_one_hot_encode_column_list, cd_dataframe_transformed_encoded, cd_features_dataframe_col_outliers_remove

    return cd_dataframe_transformed_encoded, cd_features_dataframe_col_outliers_removed
    
    

## Step 2: Feature Transformation

### Step 2.1: Apply Feature Scaling

Before we apply dimensionality reduction techniques to the data, we need to perform feature scaling so that the principal component vectors are not influenced by the natural differences in scale for features. Starting from this part of the project, you'll want to keep an eye on the [API reference page for sklearn](http://scikit-learn.org/stable/modules/classes.html) to help you navigate to all of the classes and functions that you'll need. In this substep, you'll need to check the following:

- sklearn requires that data not have missing values in order for its estimators to work properly. So, before applying the scaler to your data, make sure that you've cleaned the DataFrame of the remaining missing values. This can be as simple as just removing all data points with missing data, or applying an [Imputer](https://scikit-learn.org/0.16/modules/generated/sklearn.preprocessing.Imputer.html) to replace all missing values. You might also try a more complicated procedure where you temporarily remove missing values in order to compute the scaling parameters before re-introducing those missing values and applying imputation. Think about how much missing data you have and what possible effects each approach might have on your analysis, and justify your decision in the discussion section below.
- For the actual scaling function, a [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) instance is suggested, scaling each feature to mean 0 and standard deviation 1.
- For these classes, you can make use of the `.fit_transform()` method to both fit a procedure to the data as well as apply the transformation to the data at the same time. Don't forget to keep the fit sklearn objects handy, since you'll be applying them to the customer demographics data towards the end of the project.

In [ ]:
# If you've not yet cleaned the dataset of all NaN values, then investigate and
# do that now.

print(azdias_general_demographics_cleaned_prescaled_main_df.isnull().sum()[azdias_general_demographics_cleaned_prescaled_main_df.isnull().sum()>0])

In [ ]:
from WGU_D499_P2_DCook.eda import eda_get_missing_percentages

eda_missing_data_dataframe = eda_get_missing_percentages(azdias_general_demographics_cleaned_prescaled_main_df, features_summary_manual_df)

In [ ]:

eda_missing_data_dataframe.info()

In [ ]:
eda_missing_data_dataframe

In [ ]:
# How I'll impute


# Most frequent
#ALTERSKATEGORIE_GROB - dd 1.2 - most frequent
#HEALTH_TYP - dd 1.10 - most frequent
#VERS_TYP - dd 1.24 - most frequent
#W_KEIT_KIND_HH - dd 2.6 - most frequent
#KONSUMNAEHE - dd 3.4 - most frequent
#KBA05_ANTG1 - dd 5.1 - most frequent
#KBA05_ANTG2 - dd 5.2 - most frequent
#KBA05_ANTG3 - dd 5.3 - most frequent
#KBA05_ANTG4 - dd 5.4 - most frequent
#KBA05_GBZ - dd 5.6 - most frequent
#BALLRAUM - dd 6.1 - most frequent
#EWDICHTE - dd 6.2 - most frequent
#INNENSTADT - dd 6.3 - most frequent
#GEBAEUDETYP_RASTER - dd 7.1 - most frequent
#KKK - dd 7.2 - most frequent
#REGIOTYP - dd 7.5 - most frequent 

#PLZ8_ANTG1 - dd 8.2 - most frequent
#PLZ8_ANTG2 - dd 8.3 - most frequent
#PLZ8_ANTG3 - dd 8.4 - most frequent
#PLZ8_ANTG4 - dd 8.5 - most frequent
#PLZ8_HHZ - dd 8.7 - most frequent
#PLZ8_GBZ - dd 8.8 - most frequent
#ARBEIT - dd 9.1  - most frequent
#ORTSGR_KLS9 - dd 9.2  - most frequent
#RELAT_AB - dd 9.3  - most frequent

# Mean
#ANZ_HAUSHALTE_AKTIV - dd 3.1 - mean
#ANZ_HH_TITEL - dd 3.2 -  mean
#KBA13_ANZAHL_PKW - dd 8.1 - most frequent

In [ ]:
from WGU_D499_P2_DCook.utils import get_imput_lists

category_imputed_columns_list, numeric_imputed_columns_list, imputed_columns_list = get_imput_lists(eda_missing_data_dataframe)

In [ ]:
azdias_general_demographics_cleaned_prescaled_main_df.info()

In [ ]:
temp_azdias_general_demographics_nan_removed = azdias_general_demographics_cleaned_prescaled_main_df.dropna().reset_index(drop=True)

In [ ]:
from sklearn.impute import SimpleImputer

temp_category_imputer = SimpleImputer(strategy='most_frequent')

temp_azdias_genebral_demographics_category_imputed =  pd.DataFrame(temp_category_imputer.fit_transform(temp_azdias_general_demographics_nan_removed[category_imputed_columns_list]), columns = category_imputed_columns_list)

In [ ]:
temp_azdias_general_demographics_numeric_columns = temp_azdias_general_demographics_nan_removed[numeric_imputed_columns_list].reset_index(drop=True)

In [ ]:
temp_azdias_general_demographics_untouched_columns = temp_azdias_general_demographics_nan_removed.drop(columns=imputed_columns_list).reset_index(drop=True)

In [ ]:
temp_azdias_general_demographics_recombined = pd.concat([temp_azdias_general_demographics_untouched_columns, temp_azdias_general_demographics_numeric_columns, temp_azdias_genebral_demographics_category_imputed], axis=1)

In [ ]:
temp_azdias_general_demographics_recombined_encoded = pd.get_dummies(temp_azdias_general_demographics_recombined, columns = category_imputed_columns_list, drop_first = False)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(temp_azdias_general_demographics_recombined_encoded)

In [ ]:
# Apply feature scaling to the general population demographics data.

from sklearn.impute import SimpleImputer
number_imputer = SimpleImputer(strategy='mean')

azdias_general_demographics_numeric_imputed = pd.DataFrame(number_imputer.fit_transform(azdias_general_demographics_cleaned_prescaled_main_df[numeric_imputed_columns_list]), columns = numeric_imputed_columns_list)

In [ ]:
category_imputer = SimpleImputer(strategy='most_frequent')

azdias_genebral_demographics_category_imputed =  pd.DataFrame(category_imputer.fit_transform(azdias_general_demographics_cleaned_prescaled_main_df[category_imputed_columns_list]), columns = category_imputed_columns_list)

In [ ]:
azdias_general_demographics_untouched = azdias_general_demographics_cleaned_prescaled_main_df.drop(columns = imputed_columns_list).reset_index(drop=True)

In [ ]:
azdias_general_demographics_recombined = pd.concat([azdias_general_demographics_untouched, azdias_general_demographics_numeric_imputed, azdias_genebral_demographics_category_imputed], axis = 1)

In [ ]:
azdias_general_demographics_recombined_encoded = pd.get_dummies(azdias_general_demographics_recombined, columns = category_imputed_columns_list, drop_first = False)

In [ ]:
azdias_general_demographics_recombined_encoded = azdias_general_demographics_recombined_encoded.reindex(columns=scaler.feature_names_in_, fill_value=0)

In [ ]:
azdias_general_demographics_recombined_encoded_scaled = pd.DataFrame(scaler.transform(azdias_general_demographics_recombined_encoded), columns = scaler.feature_names_in_)

In [ ]:
azdias_general_demographics_recombined_encoded_scaled.head()

In [ ]:
azdias_general_demographics_recombined_encoded_scaled.isna().sum()

In [ ]:
write_checkpoints("interim_data", "azdias_general_demographics_recombined_encoded_scaled", azdias_general_demographics_recombined_encoded_scaled)

### Discussion 2.1: Apply Feature Scaling

(Double-click this cell and replace this text with your own text, reporting your decisions regarding feature scaling.)

### Step 2.2: Perform Dimensionality Reduction

On your scaled data, you are now ready to apply dimensionality reduction techniques.

- Use sklearn's [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class to apply principal component analysis on the data, thus finding the vectors of maximal variance in the data. To start, you should not set any parameters (so all components are computed) or set a number of components that is at least half the number of features (so there's enough features to see the general trend in variability).
- Check out the ratio of variance explained by each principal component as well as the cumulative variance explained. Try plotting the cumulative or sequential values using matplotlib's [`plot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) function. Based on what you find, select a value for the number of transformed features you'll retain for the clustering part of the project.
- Once you've made a choice for the number of components to keep, make sure you re-fit a PCA instance to perform the decided-on transformation.

In [ ]:
# Apply PCA to the data.

from sklearn.decomposition import PCA
pca = PCA()



azdias_general_demographics_main_pca = pca.fit(azdias_general_demographics_recombined_encoded_scaled)

In [ ]:
azdias_general_demographics_main_pca.explained_variance_ratio_.sum()

In [ ]:
# Investigate the variance accounted for by each principal component.

from WGU_D499_P2_DCook.plots import scree_plot_v2

scree_plot_v2(azdias_general_demographics_main_pca)

In [ ]:
from WGU_D499_P2_DCook.pca import explained_variance_component_check, do_pca_v2


num_components, final_pca, final_variance_df = explained_variance_component_check(
    dataframe=azdias_general_demographics_recombined_encoded_scaled,
    pca_function=do_pca_v2,
    explained_variance_threshold=0.90,
    plot= False
)

In [ ]:
from WGU_D499_P2_DCook.pca import auto_pick_pca_components


pca, n_components = auto_pick_pca_components(azdias_general_demographics_recombined_encoded_scaled, explained_variance_threshold=0.90)


In [ ]:
# Re-apply PCA to the data while selecting for number of components to retain.

# Generating PCA Object with the number of components to retain.
pca = PCA(n_components=n_components)

# Fitting the PCA object to the scaled data.
pca.fit(azdias_general_demographics_recombined_encoded_scaled)

# Transforming the scaled data using the fitted PCA object.
pca_data = pca.transform(azdias_general_demographics_recombined_encoded_scaled)

In [ ]:
pca_data.shape

### Discussion 2.2: Perform Dimensionality Reduction

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding dimensionality reduction. How many principal components / transformed features are you retaining for the next step of the analysis?)

### Step 2.3: Interpret Principal Components

Now that we have our transformed principal components, it's a nice idea to check out the weight of each variable on the first few components to see if they can be interpreted in some fashion.

As a reminder, each principal component is a unit vector that points in the direction of highest variance (after accounting for the variance captured by earlier principal components). The further a weight is from zero, the more the principal component is in the direction of the corresponding feature. If two features have large weights of the same sign (both positive or both negative), then increases in one tend expect to be associated with increases in the other. To contrast, features with different signs can be expected to show a negative correlation: increases in one variable should result in a decrease in the other.

- To investigate the features, you should map each weight to their corresponding feature name, then sort the features according to weight. The most interesting features for each principal component, then, will be those at the beginning and end of the sorted list. Use the data dictionary document to help you understand these most prominent features, their relationships, and what a positive or negative value on the principal component might indicate.
- You should investigate and interpret feature associations from the first three principal components in this substep. To help facilitate this, you should write a function that you can call at any time to print the sorted list of feature weights, for the *i*-th principal component. This might come in handy in the next step of the project, when you interpret the tendencies of the discovered clusters.

In [ ]:
# Map weights for the first principal component to corresponding feature names
# and then print the linked values, sorted by weight.
# HINT: Try defining a function here or in a new cell that you can reuse in the
# other cells.

from WGU_D499_P2_DCook.pca import investigate_PC

investigate_PC(pca, 0, azdias_general_demographics_recombined_encoded_scaled.columns)

In [ ]:
# Map weights for the second principal component to corresponding feature names
# and then print the linked values, sorted by weight.

investigate_PC(pca, 1, azdias_general_demographics_recombined_encoded_scaled.columns)

In [ ]:
# Map weights for the third principal component to corresponding feature names
# and then print the linked values, sorted by weight.

investigate_PC(pca, 2, azdias_general_demographics_recombined_encoded_scaled.columns)

### Discussion 2.3: Interpret Principal Components

(Double-click this cell and replace this text with your own text, reporting your observations from detailed investigation of the first few principal components generated. Can we interpret positive and negative values from them in a meaningful way?)

## Step 3: Clustering

### Step 3.1: Apply Clustering to General Population

You've assessed and cleaned the demographics data, then scaled and transformed them. Now, it's time to see how the data clusters in the principal components space. In this substep, you will apply k-means clustering to the dataset and use the average within-cluster distances from each point to their assigned cluster's centroid to decide on a number of clusters to keep.

- Use sklearn's [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) class to perform k-means clustering on the PCA-transformed data.
- Then, compute the average difference from each point to its assigned cluster's center. **Hint**: The KMeans object's `.score()` method might be useful here, but note that in sklearn, scores tend to be defined so that larger is better. Try applying it to a small, toy dataset, or use an internet search to help your understanding.
- Perform the above two steps for a number of different cluster counts. You can then see how the average distance decreases with an increasing number of clusters. However, each additional cluster provides a smaller net benefit. Use this fact to select a final number of clusters in which to group the data. **Warning**: because of the large size of the dataset, it can take a long time for the algorithm to resolve. The more clusters to fit, the longer the algorithm will take. You should test for cluster counts through at least 10 clusters to get the full picture, but you shouldn't need to test for a number of clusters above about 30.
- Once you've selected a final number of clusters to use, re-fit a KMeans instance to perform the clustering operation. Make sure that you also obtain the cluster assignments for the general demographics data, since you'll be using them in the final Step 3.3.

In [ ]:
from sklearn.cluster import KMeans

model_score=[]
clusters = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]

clusters = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]

# Over a number of different cluster counts...
for i in clusters:
    # run k-means clustering on the data and...
    kmeans = KMeans(i).fit(pca_data)
    
    # compute the average within-cluster distances.
    model_score.append(abs(kmeans.score(pca_data)))
    
    


In [ ]:
from WGU_D499_P2_DCook.clustering import compute_average_within_cluster_distance

model_score = compute_average_within_cluster_distance(pca_data, (2, 30), 2)

In [ ]:
model_score

In [ ]:
from WGU_D499_P2_DCook.clustering import get_Kmeans_scores

model_scores, cluster_range = get_Kmeans_scores(pca_data, (2, 30), 2)

In [ ]:
# Investigate the change in within-cluster distance across number of clusters.
# HINT: Use matplotlib's plot function to visualize this relationship.

from WGU_D499_P2_DCook.plots import plot_kmeans_elbow_method

plot_kmeans_elbow_method(model_scores, clusters_range=(2, 30), step_interval=2)

In [ ]:
from WGU_D499_P2_DCook.plots import plot_elbow

plot_elbow(model_scores, cluster_range)

In [ ]:
from WGU_D499_P2_DCook.clustering import find_optimal_k_knee


optimal_k = find_optimal_k_knee(model_scores, cluster_range)



In [ ]:
# Re-fit the k-means model with the selected number of clusters and obtain
# cluster predictions for the general population demographics data.

kmeans = KMeans(n_clusters = optimal_k).fit(pca_data)
kmeans_pred = kmeans.predict(pca_data)

### Discussion 3.1: Apply Clustering to General Population

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding clustering. Into how many clusters have you decided to segment the population?)

### Step 3.2: Apply All Steps to the Customer Data

Now that you have clusters and cluster centers for the general population, it's time to see how the customer data maps on to those clusters. Take care to not confuse this for re-fitting all of the models to the customer data. Instead, you're going to use the fits from the general population to clean, transform, and cluster the customer data. In the last step of the project, you will interpret how the general population fits apply to the customer data.

- Don't forget when loading in the customers data, that it is semicolon (`;`) delimited.
- Apply the same feature wrangling, selection, and engineering steps to the customer demographics using the `clean_data()` function you created earlier. (You can assume that the customer demographics data has similar meaning behind missing data patterns as the general demographics data.)
- Use the sklearn objects from the general demographics data, and apply their transformations to the customers data. That is, you should not be using a `.fit()` or `.fit_transform()` method to re-fit the old objects, nor should you be creating new sklearn objects! Carry the data through the feature scaling, PCA, and clustering steps, obtaining cluster assignments for all of the data in the customer demographics data.

In [ ]:
# Load in the customer demographics data.
customers = load_csv_with_parameters("Udacity_CUSTOMERS_Subset.csv", sep=";", na_values=np.nan)

In [ ]:
customers.shape

In [ ]:
FT_ops = [
    {"type": "map", "params": {
        "column": "OST_WEST_KZ",
        "mappings": [{"O": 1, "W": 0}, {"W": 1, "O": 0}],
        "new_names": ["WEST","EAST"],
        "pre_fill_na": True,
        "fill_na_value": '',
        "create_new": True,
        "keep_index": True,
        "keep_original": False,
        "encode_column": False
    }},
    {"type": "map", "params": {
        "column": "LP_FAMILIE_GROB",
        "mappings": [{1: 'single', 2: 'couple', 3: 'single_parent', 4: 'family', 5: 'multihousehold'}],
        "new_names": [],
        "pre_fill_na": False,
        "fill_na_value": None,
        "create_new": False,
        "keep_index": False,
        "keep_original": True,
        "encode_column": True
    }},
    {"type": "map", "params": { 
        "column": "LP_STATUS_GROB",
        "mappings": [{1: 'low_income', 2: 'avg_income', 3: 'independant', 4: 'houseowner', 5: 'top_earner'}],
        "new_names": [],
        "pre_fill_na": False,
        "fill_na_value": None,
        "create_new": False,
        "keep_index": False,
        "keep_original": True,
        "encode_column": True
    }},
    {"type": "map", "params": { 
        "column": "PRAEGENDE_JUGENDJAHRE",
        "mappings": [{1: 1, 2: 2, 3: 2, 4: 2, 5: 3, 6: 3, 7: 3, 8: 4, 9: 4, 10: 5, 11: 5, 12: 5, 13: 5, 14: 6, 15: 6}, 
                     {1: 1, 3: 1, 5: 1, 8: 1, 10: 1, 12: 1, 14: 1, 2: 2, 4: 2, 6: 2, 7: 2, 9: 2, 11: 2, 13: 2, 15: 2}],
        "new_names": ["GENERATION", "MOVEMENT"],
        "pre_fill_na": False,
        "fill_na_value": None,
        "create_new": True,
        "keep_index": True,
        "keep_original": False,
        "encode_column": True
    }},   
    {"type": "map", "params": { 
        "column": "LP_LEBENSPHASE_FEIN",
        "mappings": [{1: 'younger_age', 2: 'middle_age', 3: 'younger_age', 4: 'middle_age', 5: 'advanced_age', 6: 'retirement_age', 7: 'advanced_age', 8: 'retirement_age', 9: 'middle_age', 10: 'middle_age', 11: 'advanced_age', 12: 'retirement_age', 13: 'advanced_age', 14: 'younger_age', 15: 'advanced_age', 16: 'advanced_age', 17: 'middle_age', 18: 'younger_age', 19: 'advanced_age', 20: 'advanced_age', 21: 'middle_age', 22: 'middle_age', 23: 'middle_age', 24: 'middle_age', 25: 'middle_age', 26: 'middle_age', 27: 'middle_age', 28: 'middle_age', 29: 'younger_age', 30: 'younger_age', 31: 'advanced_age', 32: 'advanced_age',  33: 'younger_age', 34: 'younger_age', 35: 'younger_age', 36: 'advanced_age', 37: 'advanced_age', 38: 'retirement_age', 39: 'middle_age', 40: 'retirement_age'}, 
                     {1: 'single', 2: 'single', 3: 'single', 4: 'single', 5: 'single', 6: 'single', 7: 'single', 8: 'single', 9: 'single', 10: 'single', 11: 'single', 12: 'single', 13: 'single', 14: 'couples', 15: 'couples', 16: 'couples', 17: 'couples', 18: 'couples', 19: 'couples', 20: 'couples', 21: 'single_parent', 22: 'single_parent', 23: 'single_parent', 24: 'family', 25: 'family', 26: 'family', 27: 'family', 28: 'family', 29: 'multihousehold', 30: 'multihousehold', 31: 'multihousehold', 32: 'multihousehold', 33: 'multihousehold', 34: 'multihousehold', 35: 'multihousehold', 36: 'multihousehold', 37: 'multihousehold', 38: 'multihousehold', 39: 'multihousehold', 40: 'multihousehold'}, 
                     {1: 'low', 2: 'low', 3: 'average', 4: 'average', 5: 'low', 6: 'low', 7: 'average', 8: 'average', 9: 'average', 10: 'wealthy', 11: 'average', 12: 'average', 13: 'top', 14: 'average', 15: 'low', 16: 'average', 17: 'average', 18: 'wealthy', 19: 'wealthy', 20: 'top', 21: 'low', 22: 'average', 23: 'wealthy', 24: 'low', 25: 'average', 26: 'average', 27: 'average', 28: 'top', 29: 'low', 30: 'average', 31: 'low', 32: 'average', 33: 'average', 34: 'average', 35: 'top', 36: 'average', 37: 'average', 38: 'average', 39: 'top', 40: 'top'}],
        "new_names": ["LIFE_STAGE", "FAMILY_STAGE", "WEALTH_STAGE"],
        "pre_fill_na": False,
        "fill_na_value": None,
        "create_new": True,
        "keep_index": True,
        "keep_original": False,
        "encode_column": True
    }},   
    {"type": "map", "params": { 
        "column": "WOHNLAGE",
        "mappings": [{1:0, 2:0, 3:0, 4:0, 5:0, 7:1, 8:1}, 
                     {1:1, 2:1, 3:1, 4:1, 5:1, 7:0, 8:0}],
        "new_names": ["RURAL", "NEIGHBOURHOOD"],
        "pre_fill_na": True,
        "fill_na_value": 0,
        "create_new": True,
        "keep_index": True,
        "keep_original": False,
        "encode_column": False
    }},     
    {"type": "map", "params": { 
        "column": "PLZ8_BAUMAX",
        "mappings": [{1: 1, 2: 1, 3: 1, 4: 1, 5: 0}, 
                     {1: 0, 2: 0, 3: 0, 4: 0, 5: 1}],
        "new_names": ["FAMILY_BUILDING", "BUSINESS_BUILDING"],
        "pre_fill_na": True,
        "fill_na_value": 0,
        "create_new": True,
        "keep_index": True,
        "keep_original": False,
        "encode_column": False
    }},   
    {"type": "replace", "params": {
        "column": "VERS_TYP",
        "replacements": {1: 0, 2: 1}
    }},
    {"type": "replace", "params": {
        "column": "ANREDE_KZ",
        "replacements": {1: 0, 2: 1}
    }},
    {"type": "convert_data_type", "params": {
        "column": "PRAEGENDE_JUGENDJAHRE_GENERATION",
        "data_type": "category"
    }},    
    {"type": "convert_data_type", "params": {
        "column": "PRAEGENDE_JUGENDJAHRE_MOVEMENT",
        "data_type": "category"
    }},
    {"type": "convert_data_type", "params": {
        "column": "LP_LEBENSPHASE_FEIN_LIFE_STAGE",
        "data_type": "category"
    }},
    {"type": "convert_data_type", "params": {
        "column": "LP_LEBENSPHASE_FEIN_FAMILY_STAGE",
        "data_type": "category"
    }},
    {"type": "convert_data_type", "params": {
        "column": "LP_LEBENSPHASE_FEIN_WEALTH_STAGE",
        "data_type": "category"
    }},
]

In [ ]:
# Apply preprocessing, feature transformation, and clustering from the general
# demographics onto the customer data, obtaining cluster predictions for the
# customer demographics data.

from WGU_D499_P2_DCook.features import FeatureTransformer


customers_cleaned, azdias_feature_summary_auto =  clean_data(customers, features_summary_auto_df, FT_ops, FeatureTransformer)

In [ ]:
customers_cleaned.shape

In [ ]:
customers_cleaned.columns

In [ ]:
get_info_to_dataframe(customers_cleaned)

In [ ]:
get_info_to_dataframe(azdias_general_demographics_cleaned_prescaled_main_df)

In [ ]:
from WGU_D499_P2_DCook.utils import compare_dataframe_info

comparison_df = compare_dataframe_info(azdias_general_demographics_cleaned_prescaled_main_df, customers_cleaned)

In [ ]:
comparison_df

In [ ]:
# One column difference
# Customers_cleaned doesn't contained GEBAEUDETYP_5.0


In [ ]:
from WGU_D499_P2_DCook.impute_and_scale import apply_existing_scaling_v3

customers_scaled = apply_existing_scaling_v3(
    prescaled_df=customers_cleaned,
    features_df=azdias_feature_summary_auto,
    #num_imputer=num_imputer,
    #cat_imputer=cat_imputer,
    num_imputer = number_imputer,
    cat_imputer = category_imputer,
    scaler=scaler
)

In [ ]:
pca_customers = pd.DataFrame(pca.transform(customers_scaled))

In [ ]:
customer_labels = kmeans.predict(pca_customers)

In [ ]:
#customer_labels

In [ ]:
customers_scaled_with_clusters = customers_scaled.copy()

In [ ]:
customers_scaled_with_clusters['cluster'] = customer_labels

In [ ]:
print(f"General Population: \n {kmeans_pred[:7]} \n {customer_labels[:7]} \nCustomers:")

In [ ]:

customers_clusters = pd.Series(customer_labels)
azdias_clusters = pd.Series(kmeans_pred)

cluster_info = pd.DataFrame([])

cluster_info["Population"] = azdias_clusters.value_counts().sort_index()
cluster_info["Customers"] = customers_clusters.value_counts().sort_index()
cluster_info.reset_index(inplace=True)
cluster_info.rename(columns={"index":"Cluster"}, inplace=True)

In [ ]:
from WGU_D499_P2_DCook.plots import plot_cluster_distribution

plot_cluster_distribution(cluster_info)

In [ ]:
from WGU_D499_P2_DCook.plots import plot_cluster_proportions

plot_cluster_proportions(cluster_info)

### Step 3.3: Compare Customer Data to Demographics Data

At this point, you have clustered data based on demographics of the general population of Germany, and seen how the customer data for a mail-order sales company maps onto those demographic clusters. In this final substep, you will compare the two cluster distributions to see where the strongest customer base for the company is.

Consider the proportion of persons in each cluster for the general population, and the proportions for the customers. If we think the company's customer base to be universal, then the cluster assignment proportions should be fairly similar between the two. If there are only particular segments of the population that are interested in the company's products, then we should see a mismatch from one to the other. If there is a higher proportion of persons in a cluster for the customer data compared to the general population (e.g. 5% of persons are assigned to a cluster for the general population, but 15% of the customer data is closest to that cluster's centroid) then that suggests the people in that cluster to be a target audience for the company. On the other hand, the proportion of the data in a cluster being larger in the general population than the customer data (e.g. only 2% of customers closest to a population centroid that captures 6% of the data) suggests that group of persons to be outside of the target demographics.

Take a look at the following points in this step:

- Compute the proportion of data points in each cluster for the general population and the customer data. Visualizations will be useful here: both for the individual dataset proportions, but also to visualize the ratios in cluster representation between groups. Seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) or [`barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) function could be handy.
  - Recall the analysis you performed in step 1.1.3 of the project, where you separated out certain data points from the dataset if they had more than a specified threshold of missing values. If you found that this group was qualitatively different from the main bulk of the data, you should treat this as an additional data cluster in this analysis. Make sure that you account for the number of data points in this subset, for both the general population and customer datasets, when making your computations!
- Which cluster or clusters are overrepresented in the customer dataset compared to the general population? Select at least one such cluster and infer what kind of people might be represented by that cluster. Use the principal component interpretations from step 2.3 or look at additional components to help you make this inference. Alternatively, you can use the `.inverse_transform()` method of the PCA and StandardScaler objects to transform centroids back to the original data space and interpret the retrieved values directly.
- Perform a similar investigation for the underrepresented clusters. Which cluster or clusters are underrepresented in the customer dataset compared to the general population, and what kinds of people are typified by these clusters?

In [ ]:
df_population_with_clusters = azdias_general_demographics_recombined_encoded_scaled.copy()
df_population_with_clusters['cluster'] = kmeans.labels_

In [ ]:
df_population_with_clusters.info()
customers_scaled_with_clusters.info()

In [ ]:
# Compare the proportion of data in each cluster for the customer data to the
# proportion of data in each cluster for the general population.

pop_dist = df_population_with_clusters['cluster'].value_counts(normalize=True).sort_index()
cust_dist = customers_scaled_with_clusters['cluster'].value_counts(normalize=True).sort_index()

cluster_comparison_df = pd.DataFrame({
    'population': pop_dist,
    'customers': cust_dist
}).fillna(0)

cluster_comparison_df['diff'] = cluster_comparison_df['customers'] - cluster_comparison_df['population']
print(cluster_comparison_df)

In [ ]:
#df_population_with_clusters['cluster'].head(15)

In [ ]:
threshold = 0.05  # tweak as needed

underrepresented_clusters = cluster_comparison_df[cluster_comparison_df['diff'] < -threshold].index.tolist()
overrepresented_clusters = cluster_comparison_df[cluster_comparison_df['diff'] > threshold].index.tolist()

print("Underrepresented:", underrepresented_clusters)
print("Overrepresented:", overrepresented_clusters)

In [ ]:
threshold = 0.05  # tweak as needed

underrepresented_clusters = cluster_comparison_df[cluster_comparison_df['diff'] < -threshold].index.tolist()
overrepresented_clusters = cluster_comparison_df[cluster_comparison_df['diff'] > threshold].index.tolist()

print("Underrepresented:", underrepresented_clusters)
print("Overrepresented:", overrepresented_clusters)

In [ ]:
print(cluster_comparison_df.sort_values('diff'))

In [ ]:
# What kinds of people are part of a cluster that is overrepresented in the
# customer data compared to the general population?

overrepresented_clusters_v2 = cluster_comparison_df[cluster_comparison_df['diff'] > 0].index
df_over_v2 = df_population_with_clusters[df_population_with_clusters['cluster'].isin(overrepresented_clusters_v2)]

# Explore their characteristics
df_over_v2.groupby('cluster').mean(numeric_only=True)

In [ ]:
# What kinds of people are part of a cluster that is underrepresented in the
# customer data compared to the general population?

underrepresented_clusters_v2 = cluster_comparison_df[cluster_comparison_df['diff'] < 0].index
df_under_v2 = df_population_with_clusters[df_population_with_clusters['cluster'].isin(underrepresented_clusters_v2)]

# Analyze these people's characteristics
df_under_v2.groupby('cluster').mean(numeric_only=True)

### Discussion 3.3: Compare Customer Data to Demographics Data

(Double-click this cell and replace this text with your own text, reporting findings and conclusions from the clustering analysis. Can we describe segments of the population that are relatively popular with the mail-order company, or relatively unpopular with the company?)

> Congratulations on making it this far in the project! Before you finish, make sure to check through the entire notebook from top to bottom to make sure that your analysis follows a logical flow and all of your findings are documented in **Discussion** cells. Once you've checked over all of your work, you should export the notebook as an HTML document to submit for evaluation. You can do this from the menu, navigating to **File -> Download as -> HTML (.html)**. You will submit both that document and this notebook for your project submission.